In [52]:
# zadataka je da se za dat trening skup napravi term matrica
# napraviti model za klasifikaciju pomocu multinomijalnog naivnog bajesa i trening skupa
# klasifikuje test instanca X = Chinese Chinese Chinese Tokyo Japan

# imamo ugradjeno u sklearn nacin da pravimo term matricu
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

import pandas as pd


In [53]:
# ovako izgleda tabela trening skup iz zadatka, ovo je glup primer jer dokumenti 
# imaju po mali broj reci
# klasas je da li se tekst odnosi na kinu ili ne

#  id_teksta |  reci u dokumentu          | klasa
#        1   |   Chinese Beijing Chinese  | yes
#        2   |   Chinese Chinese Shanghai | yes
#        3   |   Chinese Macao            | yes
#        4   |   Tokyo Japan Chinese      | no


# term matrica je matrica gde su kolone reci, vrste su id teksta, a elementi matrice su broj pojavljivanja
# te reci u tom dokumentu, mi bismo je na papiru napravili ovako:

#  id_teksta | Chinese | Beijing | Shanghai | Macao | Tokyo | Japan
#        1   |  2      |   1     |   0      |  0    |  0    |  0
#        2   |  2      |   0     |   1      |  0    |  0    |  0
#        3   |  1      |   0     |   0      |  1    |  0    |  0
#        4   |  1      |   0     |   0      |  0    |  1    |  1


corpus = [ 'Chinese Beijing Chinese',
           'Chinese Chinese Shanghai', 
           'Chinese Macao',
           'Tokyo Japan Chinese'
         ]

classes = ['yes', 'yes', 'yes', 'no']

In [54]:
# CountVectorizer je transformacija, sve isto kao sto smo videli do sada, imamo fit i transform
# mozemo da podesavamo dosta parametara, vidi dokumentaciju. Nesto korisno sto je po defaultu True je da
# se sve reci pretvore u lowercase, da nebi imali posebne kolone za npr Chinese i chinese

# NOTE: prvo smo radili ovako sa CountVectorizer, tu bas dobijemo broj pojavljivanja svake reci u svakom tekstu
# vectorizer = CountVectorizer()

# nakon toga smo uradili ovo. Setimo se na pocetku kada smo pricali o term matricama, da nam je pozeljno da
# na neki nacin dodelimo neku vaznost svakoj reci, tj da koristimo term_frequency * log(nesto...) tj tf*idf
# ovde necemo u matrici dobiti bas broj reci, vec neki broj izmedju 0 i 1 koji kaze koliko je ta rec
# vazna za konkretan dokument. Tu je ideja bila da rec koja se cesto pojavljuje u mnogo dokumenata je manje 
# vazna nego neka specificna rec koja se malo puta pojvaljuje u nekom dokumentu (?)
vectorizer = TfidfVectorizer()

# mogli smo ovo i da izvedeomo iz dva koraka ovako:
# naravimo obicna CountVectorizer()
# na njega primenimo TfidfTransformer()

# fit i transform mozemo uraditi i ovako u jednom koraku
X_train = vectorizer.fit_transform(corpus)
# ovim pozivom se dobija nekakva sparse matrica, tj matrica za koju se zna da ce vecina elemenata biti 0
# takve matrice se pametno cuvaju u memoriji tako sto se cuvaju samo indeksi na kojima element nije 0
# zajedno sa njihovim vrednostima
print(X_train)
print(type(X_train))
# mozemo od ovoga dobiti i obicnu matricu koja je laksa za citanje
print(X_train.toarray())

  (0, 0)	0.6918346120039814
  (0, 1)	0.7220560017292982
  (1, 4)	0.6918346120039814
  (1, 1)	0.7220560017292982
  (2, 3)	0.8865476297873808
  (2, 1)	0.46263733109032296
  (3, 2)	0.6633846138519129
  (3, 5)	0.6633846138519129
  (3, 1)	0.34618161159873423
<class 'scipy.sparse.csr.csr_matrix'>
[[0.69183461 0.722056   0.         0.         0.         0.        ]
 [0.         0.722056   0.         0.         0.69183461 0.        ]
 [0.         0.46263733 0.         0.88654763 0.         0.        ]
 [0.         0.34618161 0.66338461 0.         0.         0.66338461]]


In [55]:
# ovako mozemod a vidimo sve reci koje postoje
print(vectorizer.get_feature_names())

['beijing', 'chinese', 'japan', 'macao', 'shanghai', 'tokyo']


In [56]:
# jos jedan nacin da lepo vidimo term matricu je da od nje napravimo pd.Dataframe, onda imamo lep ispis gde
# se vidi i id dokumenta i same reci i broj pojavljivanja
pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names())

,beijing,chinese,japan,macao,shanghai,tokyo
0,0.691835,0.722056,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.722056,0.000000,0.000000,0.691835,0.000000
2,0.000000,0.462637,0.000000,0.886548,0.000000,0.000000
3,0.000000,0.346182,0.663385,0.000000,0.000000,0.663385


In [57]:
# multinomijalni bajes je pogodan za podatke gde mi nesto brojimo, tj imamo frekvencije koliko se cesto 
# nesto javlja, u nasem slucaju frekvencije reci. To se lepo vidi i razume zasto radi za obican CountVecotirzer
# ali radi kako treba i za ovaj tfidf iako mozda nije intuitivno
model = MultinomialNB()

In [58]:
model.fit(X_train, classes)

MultinomialNB()

In [59]:
test_doc = 'Chinese Chinese Chinese Tokyo Japan'
# pre nego sto za nasu test instancu uradimo predict, moramo od nje napraviti oblik pogodan za model
# tj moramo i od nje da naparvimo term matricu (moramo da ga prosledimo kao listu jer transform ocekuje listu)
X_test = vectorizer.transform([test_doc])
X_test.toarray()

print(model.predict(X_test))

['yes']
